In [ ]:
import os
import sys
import math
import time
import shutil
import warnings
from pathlib import Path
import subprocess 
import cv2
import numpy as np
import torch
import easyocr
from PIL import Image, ImageDraw, ImageFont

# ============ VietOCR hiện không sử dụng, chờ fine tune ==========
# from vietocr.tool.predictor import Predictor
# from vietocr.tool.config import Cfg

from pdf2image import convert_from_path
from transformers import AutoImageProcessor, TableTransformerForObjectDetection
from ultralytics import YOLO # Giữ lại nếu YOLO_ENABLE_VALIDATION là True
from torchvision import transforms
from tqdm.auto import tqdm
import pandas as pd
from tkinter import Tk, filedialog 

warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

# --- Hàm thay thế cho cv2_imshow của Colab ---
def cv2_imshow_local(img_array, window_name="Image"):
    # Lựa chọn 1: Lưu file tạm (như code gốc)
    # cv2.imwrite("temp_display_image.png", img_array)
    # print("Ảnh được lưu vào temp_display_image.png")

    # Lựa chọn 2: Hiển thị trong cửa sổ (cần môi trường GUI)
    #cv2.imshow(window_name, img_array)
    #cv2.waitKey(1) # Đợi một chút để cửa sổ cập nhật, 0 là đợi phím bất kỳ
    return
# --- Kiểm tra Poppler ---
# pdf2image cần có poppler. Hàm này để check PATH, nếu không có thì cần add.
try:
    convert_from_path(None) # gọi hàm dummy check poppler
except Exception as e:
    if "poppler" in str(e).lower():
        print("Không có Poppler. pdf2image không hoạt động.")
        print("Cần cài Poppler và add vào PATH hệ thống.")
        sys.exit("Dừng do thiếu Poppler.") 
    # else:
    #     print(f"Lỗi không xác định với pdf2image: {e}") # Lỗi khác không phải poppler


In [ ]:
overall_start_time = time.time()
SHOW_PROCESSING_IMAGES: bool = False # Đặt False/True để hiển thị ảnh giữa process

DEFAULT_FONT = None
font_paths_to_try = ["DejaVuSans-Bold.ttf", "arial.ttf", "arialbd.ttf", "/usr/share/fonts/truetype/dejavu/DejaVuSans-Bold.ttf"]
for font_path_item in font_paths_to_try:
    try:
        DEFAULT_FONT = ImageFont.truetype(font_path_item, 12)
        break
    except IOError:
        continue
if DEFAULT_FONT is None:
    DEFAULT_FONT = ImageFont.load_default()

def display_image(title: str, image_np: np.ndarray, max_width: int = 800) -> None:
    if not SHOW_PROCESSING_IMAGES or image_np is None or image_np.size == 0:
        return
    h_orig, w_orig = image_np.shape[:2]
    display_img_resized = image_np
    if w_orig > max_width:
        scale = max_width / w_orig
        new_h = int(h_orig * scale)
        display_img_resized = cv2.resize(image_np, (max_width, new_h), interpolation=cv2.INTER_AREA)
    # print(f"\n--- Đang hiển thị: {title} (Đã thay đổi kích thước nếu >{max_width}px chiều rộng) ---")
    # cv2_imshow_local(display_img_resized, title) # Sử dụng hàm local
    # time.sleep(0.01) # Có thể không cần thiết với cv2.imshow và waitKey(1)

def display_image_with_bboxes(
    title: str, image_np: np.ndarray,
    cv_bbox_abs: tuple[int, int, int, int] | None = None,
    tt_bbox_abs: tuple[int, int, int, int] | None = None,
    max_width: int = 800
) -> None:
    if not SHOW_PROCESSING_IMAGES or image_np is None or image_np.size == 0: return
    display_img_pil = Image.fromarray(cv2.cvtColor(image_np, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(display_img_pil)
    if cv_bbox_abs:
        x, y, w, h = cv_bbox_abs
        draw.rectangle([x, y, x + w, y + h], outline="blue", width=3)
        draw.text((x + 2, y + 2), "CV Contour", fill="blue", font=DEFAULT_FONT)
    if tt_bbox_abs:
        x, y, w, h = tt_bbox_abs
        draw.rectangle([x, y, x + w, y + h], outline="red", width=3)
        draw.text((x + 2, y - 15 if y > 15 else y + h + 2), "TT Match", fill="red", font=DEFAULT_FONT)
    display_img_np_annotated = cv2.cvtColor(np.array(display_img_pil), cv2.COLOR_RGB2BGR)
    display_image(title, display_img_np_annotated, max_width)

def display_processed_crop(title: str, processed_crop_np: np.ndarray, max_width: int = 800) -> None:
    if not SHOW_PROCESSING_IMAGES or processed_crop_np is None or processed_crop_np.size == 0: return
    display_image(title, processed_crop_np, max_width)


In [ ]:
yolo_model_global: 'YOLO | None' = None
yolo_device_global: str = "cuda" if torch.cuda.is_available() else "cpu"
yolo_model_classes_global: dict[int, str] = {}

def load_yolo_model_safe(model_path_or_id: str, device: str) -> tuple['YOLO | None', dict[int, str]]:
    global yolo_model_classes_global
    loaded_model_local: 'YOLO | None' = None
    model_classes_local: dict[int, str] = {}
    try:
        from ultralytics import YOLO 
        loaded_model_local = YOLO(model_path_or_id)
        loaded_model_local.to(device)
        dummy_img = np.zeros((64, 64, 3), dtype=np.uint8)
        _ = loaded_model_local(dummy_img, device=(0 if device == 'cuda' else 'cpu'), verbose=False)
        if hasattr(loaded_model_local, 'names') and loaded_model_local.names:
            raw_names = loaded_model_local.names
            if isinstance(raw_names, dict): model_classes_local = {int(k): v for k, v in raw_names.items() if str(k).isdigit()}
            elif isinstance(raw_names, (list, tuple)): model_classes_local = {i: name for i, name in enumerate(raw_names)}
        yolo_model_classes_global = model_classes_local
    except ImportError:
        print("LỖI: Không thể nhập thư viện ultralytics. YOLO sẽ không khả dụng.")
        loaded_model_local = None
        yolo_model_classes_global = {}
    except Exception as e_yolo_load:
        print(f"LỖI: Không thể tải mô hình YOLO '{model_path_or_id}': {e_yolo_load}")
        loaded_model_local = None
        yolo_model_classes_global = {}
    return loaded_model_local, model_classes_local


In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Sử dụng thiết bị: {DEVICE}")

# --- EasyOCR Reader ---
EASYOCR_READER = None
try:
    EASYOCR_READER = easyocr.Reader(['vi'], gpu=(DEVICE == "cuda"))
except Exception as e:
    EASYOCR_READER = None

# --- Detection Model ---
TT_DETECTION_PROCESSOR = None
TT_DETECTION_MODEL = None
try:
    TT_DETECTION_MODEL_NAME = "microsoft/table-transformer-detection"
    TT_DETECTION_PROCESSOR = AutoImageProcessor.from_pretrained(TT_DETECTION_MODEL_NAME)
    TT_DETECTION_MODEL = TableTransformerForObjectDetection.from_pretrained(TT_DETECTION_MODEL_NAME)
    if TT_DETECTION_MODEL:
        TT_DETECTION_MODEL.to(DEVICE)
        TT_DETECTION_MODEL.eval()
except Exception as e:
    print(f"LỖI khi tải mô hình Table Transformer (Detection): {e}")
    TT_DETECTION_MODEL = None
    TT_DETECTION_PROCESSOR = None


TT_STRUCTURE_PROCESSOR = None 
TT_STRUCTURE_MODEL = None
try:

    TT_STRUCTURE_MODEL_NAME = "microsoft/table-structure-recognition-v1.1-all"
    TT_STRUCTURE_PROCESSOR = AutoImageProcessor.from_pretrained(TT_STRUCTURE_MODEL_NAME)
    TT_STRUCTURE_MODEL = TableTransformerForObjectDetection.from_pretrained(TT_STRUCTURE_MODEL_NAME)
    if TT_STRUCTURE_MODEL:
        TT_STRUCTURE_MODEL.to(DEVICE)
        TT_STRUCTURE_MODEL.eval()
except Exception as e:
    print(f"LỖI khi tải mô hình Table Transformer (Structure Recognition): {e}")
    TT_STRUCTURE_MODEL = None


In [ ]:

# ==============================================================================
#            *** CÁC HÀM PERSPECTIVE WARP ***
# ==============================================================================
def order_points(pts: np.ndarray) -> np.ndarray:
    rect = np.zeros((4, 2), dtype="float32"); s = pts.sum(axis=1)
    rect[0] = pts[np.argmin(s)]; rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]; rect[3] = pts[np.argmax(diff)]
    return rect

def get_largest_poly_from_mask(binary_mask_img: np.ndarray) -> np.ndarray | None:
    contours, _ = cv2.findContours(binary_mask_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours: return None
    largest_contour = max(contours, key=cv2.contourArea); peri = cv2.arcLength(largest_contour, True)
    approx_poly = cv2.approxPolyDP(largest_contour, 0.02 * peri, True)
    if len(approx_poly) == 4: return np.squeeze(approx_poly)
    else:
        rect = cv2.minAreaRect(largest_contour)
        box_points = cv2.boxPoints(rect)
        return np.int0(box_points)

def get_max_dimensions(point_array: np.ndarray) -> tuple[int, int]:
    if point_array is None or len(point_array) != 4: return 0, 0
    rect_ordered = order_points(point_array.astype(np.float32)); (tl, tr, br, bl) = rect_ordered
    width_a = np.sqrt(((br[0] - bl[0])**2) + ((br[1] - bl[1])**2))
    width_b = np.sqrt(((tr[0] - tl[0])**2) + ((tr[1] - tl[1])**2))
    max_width = max(int(width_a), int(width_b))
    height_a = np.sqrt(((tr[0] - br[0])**2) + ((tr[1] - br[1])**2))
    height_b = np.sqrt(((tl[0] - bl[0])**2) + ((tl[1] - bl[1])**2))
    max_height = max(int(height_a), int(height_b))
    return max_width, max_height

def get_perspective_transform_matrix(point_array: np.ndarray, target_width: int, target_height: int) -> np.ndarray | None:
    if point_array is None or len(point_array) != 4 or target_width <= 0 or target_height <= 0: return None
    src_pts_ordered = order_points(point_array.astype(np.float32))
    dst_pts_ordered = np.float32([[0,0],[target_width-1,0],[target_width-1,target_height-1],[0,target_height-1]])
    try: return cv2.getPerspectiveTransform(src_pts_ordered, dst_pts_ordered)
    except cv2.error: return None

def warp_image_from_mask(image_bgr: np.ndarray, binary_mask_img: np.ndarray) -> np.ndarray:
    if image_bgr is None or image_bgr.size==0 or binary_mask_img is None or binary_mask_img.size==0: return image_bgr
    poly_coords_from_mask = get_largest_poly_from_mask(binary_mask_img)
    if poly_coords_from_mask is None or len(poly_coords_from_mask) != 4: return image_bgr
    max_w_target, max_h_target = get_max_dimensions(poly_coords_from_mask)
    if max_w_target <= 10 or max_h_target <= 10: return image_bgr
    transform_matrix_calc = get_perspective_transform_matrix(poly_coords_from_mask, max_w_target, max_h_target)
    if transform_matrix_calc is None: return image_bgr
    try:
        warped_image_result = cv2.warpPerspective(image_bgr, transform_matrix_calc, (max_w_target, max_h_target), flags=cv2.INTER_LANCZOS4)
        return warped_image_result if warped_image_result is not None and warped_image_result.size > 0 else image_bgr
    except Exception: return image_bgr


In [ ]:

# ==============================================================================
#                      *** Xoay, Chỉnh nghiêng trang ***
# ==============================================================================
def rotate_image_safe(image: np.ndarray, angle: float,
    border_val: tuple[int, int, int] | int | None = None,
    border_mode: int = cv2.BORDER_REPLICATE
) -> np.ndarray:
    if image is None or image.size == 0 or abs(angle) < 0.01:
        return image
    try:
        h, w = image.shape[:2]; center = (w // 2, h // 2)
        rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
        if border_val is None: border_val = (255,255,255) if image.ndim==3 else 255
        rotated_image = cv2.warpAffine(image,rotation_matrix,(w,h),flags=cv2.INTER_LANCZOS4,borderMode=border_mode,borderValue=border_val)
        return rotated_image if rotated_image is not None and rotated_image.size > 0 else image
    except Exception as e_rotate:
        print(f"Cảnh báo: Lỗi trong quá trình xoay ảnh: {e_rotate}")
        return image

def deskew_page_basic(image_bgr: np.ndarray, angle_thresh_deg: float = 1.0,
    bg_color: tuple[int, int, int] = (255, 255, 255)
) -> np.ndarray:
    if image_bgr is None or image_bgr.size == 0: return image_bgr
    original_image_copy = image_bgr.copy(); h_orig, w_orig = image_bgr.shape[:2]
    try:
        gray_img = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
        try: thresh_inv_img = cv2.adaptiveThreshold(gray_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY_INV,21,10)
        except cv2.error: _, thresh_inv_img = cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY_INV|cv2.THRESH_OTSU)
        kernel_width = max(15,int(w_orig*0.04)); kernel_height = max(3,int(h_orig*0.005))
        morph_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_width,kernel_height))
        dilated_img = cv2.dilate(thresh_inv_img,morph_kernel,iterations=2)
        edges_img = cv2.Canny(dilated_img,50,150,apertureSize=3)
        if cv2.countNonZero(edges_img)==0: return original_image_copy
        hough_line_thresh=max(50,int(w_orig*0.1)); min_line_length_hough=max(40,int(w_orig*0.08)); max_line_gap_hough=max(10,int(w_orig*0.02))
        lines_detected = cv2.HoughLinesP(edges_img,1,np.pi/180,hough_line_thresh,minLineLength=min_line_length_hough,maxLineGap=max_line_gap_hough)
        if lines_detected is None or len(lines_detected)==0: return original_image_copy
        detected_angles_deg = [math.degrees(math.atan2(line[0][3]-line[0][1],line[0][2]-line[0][0]))
                               for line in lines_detected
                               if line[0][2]!=line[0][0] and abs(math.degrees(math.atan2(line[0][3]-line[0][1],line[0][2]-line[0][0])))<30.0]
        if not detected_angles_deg: return original_image_copy
        median_detected_angle = float(np.median(detected_angles_deg))
        if not np.isfinite(median_detected_angle): return original_image_copy
        correction_angle_val = -median_detected_angle
        if abs(correction_angle_val) >= angle_thresh_deg:
            deskewed_img_result = rotate_image_safe(original_image_copy, correction_angle_val, border_val=bg_color)
            return deskewed_img_result
        return original_image_copy
    except Exception as e_deskew_page:
        print(f"Cảnh báo: Lỗi trong quá trình chỉnh nghiêng trang: {e_deskew_page}")
        return original_image_copy


In [ ]:
# Phần này chỉnh nghiêng bảng tuy nhiên chưa xử lý chưa xử lý perspective
def deskew_table_precisely_v2(image_crop_bgr: np.ndarray, **dk_params) -> tuple[np.ndarray, float]:
    if image_crop_bgr is None or image_crop_bgr.size==0: return image_crop_bgr,0.0
    original_crop_copy = image_crop_bgr.copy(); h_crop,w_crop = original_crop_copy.shape[:2]
    if h_crop<10 or w_crop<10: return original_crop_copy,0.0
    applied_correction_angle=0.0; params=dk_params
    try:
        gray_crop = cv2.cvtColor(original_crop_copy,cv2.COLOR_BGR2GRAY)
        try: bin_inv_crop = cv2.adaptiveThreshold(~gray_crop,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,-2)
        except cv2.error: _,bin_inv_crop = cv2.threshold(~gray_crop,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
        morph_kernel_w_factor = max(3,int(w_crop*params.get('morph_kernel_width_factor',0.15)))
        h_line_kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(morph_kernel_w_factor,1))
        h_lines_mask = cv2.morphologyEx(bin_inv_crop,cv2.MORPH_OPEN,h_line_kernel,iterations=params.get('morph_iterations',1))
        if cv2.countNonZero(h_lines_mask)<(w_crop*0.05): return original_crop_copy,0.0
        edges_from_h_mask = cv2.Canny(h_lines_mask,params.get('canny_low_thresh',50),params.get('canny_high_thresh',150),apertureSize=3)
        if cv2.countNonZero(edges_from_h_mask)==0: return original_crop_copy,0.0
        table_lines = cv2.HoughLinesP(edges_from_h_mask,1,np.pi/180,
            threshold=params.get('hough_threshold',25),
            minLineLength=max(15,int(w_crop*params.get('hough_min_line_length_factor',0.2))),
            maxLineGap=max(5,int(w_crop*params.get('hough_max_line_gap_factor',0.05))) )
        if table_lines is None or len(table_lines)==0: return original_crop_copy,0.0
        angle_filter_degrees_param = params.get('angle_filter_degrees',25.0)
        h_detected_angles = [math.degrees(math.atan2(l_seg[0][3]-l_seg[0][1],l_seg[0][2]-l_seg[0][0]))
                             for l_seg in table_lines
                             if l_seg[0][2]!=l_seg[0][0] and abs(math.degrees(math.atan2(l_seg[0][3]-l_seg[0][1],l_seg[0][2]-l_seg[0][0])))<=angle_filter_degrees_param]
        if not h_detected_angles: return original_crop_copy,0.0
        median_horizontal_angle = float(np.median(h_detected_angles))
        if not np.isfinite(median_horizontal_angle): return original_crop_copy,0.0
        h_correction_angle = -median_horizontal_angle
        if abs(h_correction_angle)>params.get('rotation_threshold_degrees',0.2):
            rotated_table_crop = rotate_image_safe(original_crop_copy,h_correction_angle)
            applied_correction_angle = h_correction_angle
            return rotated_table_crop, applied_correction_angle
        return original_crop_copy,0.0
    except Exception as e_deskew_table:
        print(f"Cảnh báo: Lỗi trong quá trình chỉnh nghiêng bảng chính xác: {e_deskew_table}")
        return original_crop_copy,0.0


In [ ]:

# ==============================================================================
#                       *** XỬ LÝ BẢNG ***
# ==============================================================================
def calculate_iou_xywh(box1_xywh:tuple[int,int,int,int],box2_xywh:tuple[int,int,int,int])->float:
    x1_i,y1_i,w1_i,h1_i=box1_xywh; x2_i,y2_i,w2_i,h2_i=box2_xywh
    b1_x_min,b1_y_min,b1_x_max,b1_y_max = x1_i,y1_i,x1_i+w1_i,y1_i+h1_i
    b2_x_min,b2_y_min,b2_x_max,b2_y_max = x2_i,y2_i,x2_i+w2_i,y2_i+h2_i
    inter_x_min=max(b1_x_min,b2_x_min); inter_y_min=max(b1_y_min,b2_y_min)
    inter_x_max=min(b1_x_max,b2_x_max); inter_y_max=min(b1_y_max,b2_y_max)
    if inter_x_max<inter_x_min or inter_y_max<inter_y_min: return 0.0
    intersection_area=(inter_x_max-inter_x_min)*(inter_y_max-inter_y_min)
    b1_area=w1_i*h1_i; b2_area=w2_i*h2_i
    union_area=float(b1_area+b2_area-intersection_area)
    if union_area <= 0: return 0.0 if intersection_area == 0 else 1.0
    return intersection_area/union_area

def detect_tables_cv_tt_combined(
    page_image_path:str|Path, output_dir_prefix:str,
    cv_detect_params:dict, table_deskew_params:dict,
    iou_match_threshold:float=0.4, tt_page_confidence_threshold:float=0.6,
    # Truyền các model đã load vào
    tt_detection_model: 'TableTransformerForObjectDetection',
    tt_detection_processor: 'AutoImageProcessor',
    tt_structure_model: 'TableTransformerForObjectDetection',
    # tt_structure_processor: 'AutoImageProcessor', # Có thể dùng chung processor
    easyocr_reader: 'easyocr.Reader'
) -> list[str]:
    global TT_DETECTION_MODEL, TT_DETECTION_PROCESSOR, TT_STRUCTURE_MODEL, EASYOCR_READER, DEVICE

    if not TT_DETECTION_MODEL or not TT_DETECTION_PROCESSOR or \
       not TT_STRUCTURE_MODEL or not EASYOCR_READER:
        print("Lỗi: Một hoặc nhiều model cần thiết (TT Detection, TT Structure, EasyOCR) chưa được tải.")
        return []

    saved_crop_paths_list:list[str]=[]; page_file_path=Path(page_image_path); page_base_name=page_file_path.name
    img_page_bgr_original_data=cv2.imread(str(page_file_path))
    if img_page_bgr_original_data is None: print(f"Lỗi: Không thể đọc ảnh trang: {page_file_path}"); return []

    page_height,page_width = img_page_bgr_original_data.shape[:2]
    if page_height<=10 or page_width<=10: print(f"Lỗi: Kích thước ảnh trang quá nhỏ: {page_file_path}"); return []
    # print(f"\n--- Đang xử lý trang: {page_base_name} ---")

    cv_candidate_bboxes_xywh_list:list[tuple[int,int,int,int]]=[]
    try:
        dt_params_cv=cv_detect_params; gray_page_img=cv2.cvtColor(img_page_bgr_original_data,cv2.COLOR_BGR2GRAY)
        try:thresh_inv_page_img=cv2.adaptiveThreshold(~gray_page_img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,cv2.THRESH_BINARY,15,-2)
        except:_,thresh_inv_page_img=cv2.threshold(~gray_page_img,0,255,cv2.THRESH_BINARY|cv2.THRESH_OTSU)
        kernel_h_width=max(20,int(page_width*dt_params_cv.get('min_line_ratio',0.02)))
        kernel_v_height=max(20,int(page_height*dt_params_cv.get('min_line_ratio',0.02)))
        kernel_h_lines=cv2.getStructuringElement(cv2.MORPH_RECT,(kernel_h_width,1))
        kernel_v_lines=cv2.getStructuringElement(cv2.MORPH_RECT,(1,kernel_v_height))
        opened_h_lines_mask=cv2.morphologyEx(thresh_inv_page_img,cv2.MORPH_OPEN,kernel_h_lines)
        opened_v_lines_mask=cv2.morphologyEx(thresh_inv_page_img,cv2.MORPH_OPEN,kernel_v_lines)
        combined_lines_mask=cv2.addWeighted(opened_h_lines_mask,0.5,opened_v_lines_mask,0.5,0.0)
        dilate_iterations_cv=dt_params_cv.get('dilate_iter',2)
        if dilate_iterations_cv > 0:
            final_cv_mask = cv2.dilate(combined_lines_mask,cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3)),iterations=dilate_iterations_cv)
        else:
            final_cv_mask = combined_lines_mask
        if final_cv_mask is None or cv2.countNonZero(final_cv_mask)==0: raise ValueError("Mặt nạ OpenCV rỗng.")
        cv_contours_found,_=cv2.findContours(final_cv_mask,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        if not cv_contours_found: raise ValueError("Không tìm thấy đường viền nào bởi OpenCV.")
        min_area_pixels_cv=dt_params_cv.get('min_area_ratio',0.008)*page_width*page_height
        min_dimension_pixels_cv=dt_params_cv.get('min_dim_px',25)
        candidate_contours_data_list = []
        for c_item in cv_contours_found:
            x_c, y_c, w_c, h_c = cv2.boundingRect(c_item)
            area_c = cv2.contourArea(c_item)
            if area_c > min_area_pixels_cv and w_c > min_dimension_pixels_cv and h_c > min_dimension_pixels_cv:
                candidate_contours_data_list.append({'bbox_xywh': (x_c, y_c, w_c, h_c), 'area': area_c})
        if not candidate_contours_data_list: raise ValueError("Không có đường viền OpenCV nào vượt qua bộ lọc kích thước/diện tích.")
        candidate_contours_data_list.sort(key=lambda item:item['area'],reverse=True)
        selected_indices_cv=set(); final_cv_candidates_list=[]
        overlap_threshold_cv=dt_params_cv.get('overlap_threshold_ratio',0.7)
        for i_idx in range(len(candidate_contours_data_list)):
            if i_idx in selected_indices_cv: continue
            current_contour_data=candidate_contours_data_list[i_idx]
            final_cv_candidates_list.append(current_contour_data['bbox_xywh'])
            for j_idx in range(i_idx + 1, len(candidate_contours_data_list)):
                if j_idx in selected_indices_cv: continue
                other_contour_data=candidate_contours_data_list[j_idx]
                xA_overlap=max(current_contour_data['bbox_xywh'][0],other_contour_data['bbox_xywh'][0])
                yA_overlap=max(current_contour_data['bbox_xywh'][1],other_contour_data['bbox_xywh'][1])
                xB_overlap=min(current_contour_data['bbox_xywh'][0]+current_contour_data['bbox_xywh'][2],other_contour_data['bbox_xywh'][0]+other_contour_data['bbox_xywh'][2])
                yB_overlap=min(current_contour_data['bbox_xywh'][1]+current_contour_data['bbox_xywh'][3],other_contour_data['bbox_xywh'][1]+other_contour_data['bbox_xywh'][3])
                intersection_area_val=max(0,xB_overlap-xA_overlap)*max(0,yB_overlap-yA_overlap)
                if intersection_area_val > 0 and (intersection_area_val/min(current_contour_data['area'],other_contour_data['area'])) > overlap_threshold_cv:
                    selected_indices_cv.add(j_idx)
        cv_candidate_bboxes_xywh_list = final_cv_candidates_list
        print(f"  > Phát hiện OpenCV: Tìm thấy {len(cv_candidate_bboxes_xywh_list)} vùng ứng viên sau khi lọc.")
    except Exception as e_cv_detect: print(f"  > Cảnh báo: Lỗi giai đoạn phát hiện OpenCV: {e_cv_detect}"); cv_candidate_bboxes_xywh_list=[]

    tt_detected_bboxes_on_page_xywh_list:list[tuple[int,int,int,int]]=[]
    if TT_DETECTION_MODEL and TT_DETECTION_PROCESSOR:
        try:
            pil_page_img = Image.fromarray(cv2.cvtColor(img_page_bgr_original_data,cv2.COLOR_BGR2RGB))
            tt_inputs = TT_DETECTION_PROCESSOR(images=pil_page_img,return_tensors="pt")
            tt_model_device = next(TT_DETECTION_MODEL.parameters()).device
            tt_inputs = {k_item:v_item.to(tt_model_device) for k_item,v_item in tt_inputs.items()}
            with torch.no_grad(): tt_outputs = TT_DETECTION_MODEL(**tt_inputs)
            tt_target_sizes=torch.tensor([pil_page_img.size[::-1]],device=tt_model_device)
            tt_results_list = TT_DETECTION_PROCESSOR.post_process_object_detection(
                tt_outputs, threshold=tt_page_confidence_threshold, target_sizes=tt_target_sizes
            )[0]
            min_dim_pixels_tt = cv_detect_params.get('min_dim_px',25)
            for score_val,label_val,box_xyxy_val in zip(tt_results_list["scores"],tt_results_list["labels"],tt_results_list["boxes"]):
                if "table" in TT_DETECTION_MODEL.config.id2label[label_val.item()].lower():
                    xmin_val,ymin_val,xmax_val,ymax_val=[int(round(c_coord.item())) for c_coord in box_xyxy_val]
                    w_val,h_val=xmax_val-xmin_val,ymax_val-ymin_val
                    if w_val > min_dim_pixels_tt and h_val > min_dim_pixels_tt:
                        tt_detected_bboxes_on_page_xywh_list.append((xmin_val,ymin_val,w_val,h_val))
            print(f"  > Table Transformer (Cấp trang): Tìm thấy {len(tt_detected_bboxes_on_page_xywh_list)} "
                  f"vùng (độ tin cậy > {tt_page_confidence_threshold}).")
        except Exception as e_tt_page_detect: print(f"  > Cảnh báo: Lỗi phát hiện cấp trang Table Transformer: {e_tt_page_detect}"); tt_detected_bboxes_on_page_xywh_list=[]
    else:
        print("  > Mô hình Table Transformer Detection không khả dụng. Bỏ qua phát hiện cấp trang TT.")

    final_confirmed_pairs_data_list = []
    if cv_candidate_bboxes_xywh_list and tt_detected_bboxes_on_page_xywh_list:
        print(f"  > So sánh: Đang so sánh {len(cv_candidate_bboxes_xywh_list)} bboxes CV với "
              f"{len(tt_detected_bboxes_on_page_xywh_list)} bboxes TT...")
        sorted_cv_bboxes_list = sorted(cv_candidate_bboxes_xywh_list,key=lambda b_item:(b_item[1],b_item[0]))
        for cv_bbox_item in sorted_cv_bboxes_list:
            best_iou_for_this_cv_bbox = 0.0; best_tt_match_for_this_cv_bbox = None
            for tt_bbox_item in tt_detected_bboxes_on_page_xywh_list:
                iou_val = calculate_iou_xywh(cv_bbox_item, tt_bbox_item)
                if iou_val > best_iou_for_this_cv_bbox:
                    best_iou_for_this_cv_bbox = iou_val
                    best_tt_match_for_this_cv_bbox = tt_bbox_item
            if best_iou_for_this_cv_bbox > iou_match_threshold and best_tt_match_for_this_cv_bbox is not None:
                final_confirmed_pairs_data_list.append({
                    "cv_bbox": cv_bbox_item, "tt_bbox": best_tt_match_for_this_cv_bbox, "iou": best_iou_for_this_cv_bbox
                })
        print(f"    - Tìm thấy {len(final_confirmed_pairs_data_list)} cặp CV-TT có IoU > {iou_match_threshold}.")

    if not final_confirmed_pairs_data_list:
        print(f"  > So sánh: Không có bảng nào vượt qua ngưỡng khớp IoU {iou_match_threshold}. Không có crop nào để xử lý.")
        return []
    print(f"  > So sánh: {len(final_confirmed_pairs_data_list)} bboxes CV được xác nhận bởi TT. Đang tiến hành xử lý...")

    processed_table_count = 0
    for pair_index, pair_item_data in enumerate(final_confirmed_pairs_data_list):
        cv_bbox_to_process_item = pair_item_data["cv_bbox"]
        matched_tt_bbox_absolute = pair_item_data["tt_bbox"]
        iou_score_val = pair_item_data["iou"]

        if SHOW_PROCESSING_IMAGES:
            display_image_with_bboxes(
                title=f"Trang {page_base_name} - Khớp {pair_index+1} (IoU {iou_score_val:.2f})",
                image_np=img_page_bgr_original_data,
                cv_bbox_abs=cv_bbox_to_process_item, tt_bbox_abs=matched_tt_bbox_absolute
            )
        x_cv_coord, y_cv_coord, w_cv_dim, h_cv_dim = cv_bbox_to_process_item
        padding_val = cv_detect_params.get('padding',5)
        y1_crop_coord = max(0, y_cv_coord - padding_val); y2_crop_coord = min(page_height, y_cv_coord + h_cv_dim + padding_val)
        x1_crop_coord = max(0, x_cv_coord - padding_val); x2_crop_coord = min(page_width, x_cv_coord + w_cv_dim + padding_val)

        if (x2_crop_coord - x1_crop_coord) <= 10 or (y2_crop_coord - y1_crop_coord) <= 10:
            print(f"    - Xử lý: CV crop {pair_index+1} quá nhỏ. Bỏ qua."); continue
        cv_crop_bgr_padded_img = img_page_bgr_original_data[y1_crop_coord:y2_crop_coord, x1_crop_coord:x2_crop_coord].copy()
        if cv_crop_bgr_padded_img is None or cv_crop_bgr_padded_img.size == 0:
            print(f"    - Xử lý: CV crop rỗng {pair_index+1}. Bỏ qua."); continue

        deskewed_table_crop_img = cv_crop_bgr_padded_img.copy()
        try:
            deskewed_output_img, applied_angle_val = deskew_table_precisely_v2(cv_crop_bgr_padded_img, **table_deskew_params)
            if deskewed_output_img is not None and deskewed_output_img.size > 0:
                deskewed_table_crop_img = deskewed_output_img
                if abs(applied_angle_val) > 0.01: print(f"      - Đã áp dụng Chỉnh nghiêng chính xác: {applied_angle_val:.2f}°")
        except Exception as e_deskew_crop: print(f"    - Cảnh báo: Lỗi chỉnh nghiêng bảng chính xác cho crop {pair_index+1}: {e_deskew_crop}")

        final_processed_crop_for_saving_img = deskewed_table_crop_img.copy()
        try:
            if deskewed_table_crop_img is not None and deskewed_table_crop_img.size > 0:
                gray_for_warp_mask_img = cv2.cvtColor(deskewed_table_crop_img, cv2.COLOR_BGR2GRAY)
                _, bin_for_warp_mask_img = cv2.threshold(gray_for_warp_mask_img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
                warped_output_img = warp_image_from_mask(deskewed_table_crop_img, bin_for_warp_mask_img)
                if warped_output_img is not None and warped_output_img.size > 0 and \
                   warped_output_img.shape[0] > 10 and warped_output_img.shape[1] > 10:
                    final_processed_crop_for_saving_img = warped_output_img
        except Exception as e_warp_crop: print(f"    - Cảnh báo: Lỗi perspective warp cho crop {pair_index+1}: {e_warp_crop}")

        if final_processed_crop_for_saving_img is not None and final_processed_crop_for_saving_img.size > 0:
            processed_table_count += 1
            if SHOW_PROCESSING_IMAGES:
                display_processed_crop(
                    title=f"Trang {page_base_name} - CV-Crop đã xử lý {processed_table_count} (IoU {iou_score_val:.2f})",
                    processed_crop_np=final_processed_crop_for_saving_img
                )
            
            # --- Chuyển sang PIL Image để xử lý cấu trúc ---
            # Ảnh đầu vào cho structure model là final_processed_crop_for_saving_img
            h_orig_crop, w_orig_crop = final_processed_crop_for_saving_img.shape[:2]
            
            # Resize nếu quá lớn cho model structure (thường có giới hạn kích thước đầu vào)
            # MaxResize class và structure_transform vẫn giữ nguyên như code gốc
            class MaxResize(object):
                def __init__(self, max_size=1000): # Giữ nguyên 1000 hoặc điều chỉnh
                    self.max_size = max_size
                def __call__(self, image_pil):
                    width, height = image_pil.size
                    current_max_size = max(width, height)
                    if current_max_size > self.max_size:
                        scale = self.max_size / current_max_size
                        return image_pil.resize((int(round(scale * width)), int(round(scale * height))))
                    return image_pil

            structure_transform = transforms.Compose([
                MaxResize(1000), # Áp dụng MaxResize
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
            
            pil_crop_image_for_structure = Image.fromarray(cv2.cvtColor(final_processed_crop_for_saving_img, cv2.COLOR_BGR2RGB))
            
            # Lưu kích thước gốc của ảnh PIL *trước khi* transform để rescale bbox sau này
            pil_crop_image_for_structure_resized = MaxResize(1000)(pil_crop_image_for_structure.copy()) # Get the size after potential resize
            original_pil_size_for_rescale = pil_crop_image_for_structure_resized.size


            pixel_values = structure_transform(pil_crop_image_for_structure).unsqueeze(0).to(DEVICE)

            with torch.no_grad():
                outputs_structure = TT_STRUCTURE_MODEL(pixel_values)

            def box_cxcywh_to_xyxy(x):
                x_c, y_c, w, h = x.unbind(-1)
                b = [(x_c - 0.5 * w), (y_c - 0.5 * h), (x_c + 0.5 * w), (y_c + 0.5 * h)]
                return torch.stack(b, dim=1)

            def rescale_bboxes(out_bbox, size): # size là (width, height) của ảnh đầu vào cho model
                img_w, img_h = size
                b = box_cxcywh_to_xyxy(out_bbox)
                b = b * torch.tensor([img_w, img_h, img_w, img_h], dtype=torch.float32, device=out_bbox.device) # device match
                return b

            def outputs_to_objects(outputs, img_size, id2label_map): # img_size là kích thước ảnh mà model thấy
                m = outputs.logits.softmax(-1).max(-1)
                pred_labels = list(m.indices.detach().cpu().numpy())[0]
                pred_scores = list(m.values.detach().cpu().numpy())[0]
                pred_bboxes_normalized = outputs['pred_boxes'].detach().cpu()[0] # Bboxes này là normalized [0,1] cxcywh
                
                # Rescale bboxes về kích thước ảnh đầu vào của model structure (sau MaxResize)
                pred_bboxes_rescaled_xyxy = [elem.tolist() for elem in rescale_bboxes(pred_bboxes_normalized, img_size)]

                objects = []
                for label, score, bbox_xyxy in zip(pred_labels, pred_scores, pred_bboxes_rescaled_xyxy):
                    class_label = id2label_map[int(label)]
                    if class_label != "no object": # và có thể thêm score threshold
                        objects.append({'label': class_label, 'score': float(score), 
                                        'bbox_xyxy_rescaled': bbox_xyxy}) # Lưu bbox đã rescale
                return objects

            structure_id2label = TT_STRUCTURE_MODEL.config.id2label
            structure_id2label[len(structure_id2label)] = "no object" # Thêm nhãn "no object"
            
            # Sử dụng original_pil_size_for_rescale (kích thước ảnh sau MaxResize)
            cells_detected_structure = outputs_to_objects(outputs_structure, original_pil_size_for_rescale, structure_id2label)


            def get_cell_coordinates_by_row(table_data_structure, img_pil_for_ocr: Image.Image):
                # table_data_structure chứa các 'bbox_xyxy_rescaled'
                # img_pil_for_ocr là ảnh PIL mà từ đó các cell sẽ được crop (ảnh gốc của crop table, trước khi ToTensor/Normalize)
                
                rows = [entry for entry in table_data_structure if entry['label'] == 'table row']
                columns = [entry for entry in table_data_structure if entry['label'] == 'table column']
                
                # Sắp xếp rows theo tọa độ y_min, columns theo x_min
                rows.sort(key=lambda x: x['bbox_xyxy_rescaled'][1])
                columns.sort(key=lambda x: x['bbox_xyxy_rescaled'][0])

                # Bỏ các cột/hàng chồng lấn quá nhiều hoặc không hợp lệ (tùy chọn, có thể phức tạp)
                # Ví dụ: nếu 2 cột có x_min quá gần nhau, có thể là lỗi phát hiện

                cell_coordinates_extracted = []
                img_w_ocr, img_h_ocr = img_pil_for_ocr.size # Kích thước của ảnh gốc dùng để crop OCR

                # Tính scale factor nếu kích thước ảnh dùng cho model structure khác với ảnh OCR
                # original_pil_size_for_rescale là (width, height) của ảnh sau MaxResize (mà model structure thấy)
                scale_x = img_w_ocr / original_pil_size_for_rescale[0]
                scale_y = img_h_ocr / original_pil_size_for_rescale[1]


                for row_info in rows:
                    row_cells_info = []
                    y_min_row, y_max_row = row_info['bbox_xyxy_rescaled'][1], row_info['bbox_xyxy_rescaled'][3]

                    for col_info in columns:
                        x_min_col, x_max_col = col_info['bbox_xyxy_rescaled'][0], col_info['bbox_xyxy_rescaled'][2]
                        
                        # Tạo bounding box cho cell từ giao của row và column
                        # Đây là tọa độ trên ảnh đã qua MaxResize
                        cell_bbox_rescaled_xyxy = [x_min_col, y_min_row, x_max_col, y_max_row]

                        # Chuyển đổi bbox này về tọa độ của ảnh gốc (img_pil_for_ocr) để crop
                        # bbox for cropping on the original image
                        # Tọa độ cell trên ảnh gốc (dùng để crop cho OCR)
                        final_crop_coords_on_original_img = [
                            int(cell_bbox_rescaled_xyxy[0] * scale_x),
                            int(cell_bbox_rescaled_xyxy[1] * scale_y),
                            int(cell_bbox_rescaled_xyxy[2] * scale_x),
                            int(cell_bbox_rescaled_xyxy[3] * scale_y)
                        ]
                        # Đảm bảo tọa độ nằm trong ảnh
                        final_crop_coords_on_original_img[0] = max(0, final_crop_coords_on_original_img[0])
                        final_crop_coords_on_original_img[1] = max(0, final_crop_coords_on_original_img[1])
                        final_crop_coords_on_original_img[2] = min(img_w_ocr, final_crop_coords_on_original_img[2])
                        final_crop_coords_on_original_img[3] = min(img_h_ocr, final_crop_coords_on_original_img[3])


                        # Chỉ thêm cell nếu nó có kích thước hợp lệ
                        if final_crop_coords_on_original_img[2] > final_crop_coords_on_original_img[0] and \
                           final_crop_coords_on_original_img[3] > final_crop_coords_on_original_img[1]:
                            row_cells_info.append({
                                'column_bbox_rescaled': col_info['bbox_xyxy_rescaled'], # Bbox cột trên ảnh rescaled
                                'cell_bbox_for_ocr': final_crop_coords_on_original_img # Bbox cell trên ảnh gốc để OCR
                            })
                    
                    # Sắp xếp các cell trong hàng theo tọa độ x_min của cột
                    row_cells_info.sort(key=lambda x: x['column_bbox_rescaled'][0])
                    if row_cells_info: # Chỉ thêm hàng nếu có cell
                        cell_coordinates_extracted.append({
                            'row_bbox_rescaled': row_info['bbox_xyxy_rescaled'], # Bbox hàng trên ảnh rescaled
                            'cells': row_cells_info,
                            'cell_count': len(row_cells_info)
                        })
                
                # Sắp xếp các hàng theo tọa độ y_min của hàng
                cell_coordinates_extracted.sort(key=lambda x: x['row_bbox_rescaled'][1])
                return cell_coordinates_extracted

            # Sử dụng pil_crop_image_for_structure (ảnh PIL gốc của crop, trước khi ToTensor/Normalize) để OCR
            cell_coords_for_ocr = get_cell_coordinates_by_row(cells_detected_structure, pil_crop_image_for_structure)

            def apply_ocr_to_cells(cell_coordinates_input, ocr_reader, image_to_crop_from: Image.Image):
                ocr_data = dict()
                max_cols = 0
                if not ocr_reader:
                    print("Lỗi: EasyOCR Reader không khả dụng cho việc OCR cell.")
                    return {}, 0
                
                for r_idx, row_data_item in enumerate(tqdm(cell_coordinates_input, desc="OCR Cells")):
                    current_row_texts = []
                    for cell_item in row_data_item["cells"]:
                        # cell_item["cell_bbox_for_ocr"] là tọa độ [x_min, y_min, x_max, y_max] trên image_to_crop_from
                        cell_bbox = cell_item["cell_bbox_for_ocr"]
                        
                        # Crop cell từ image_to_crop_from (ảnh PIL gốc của table crop)
                        cropped_cell_pil = image_to_crop_from.crop(cell_bbox)
                        cropped_cell_np = np.array(cropped_cell_pil) # Chuyển sang Numpy array cho EasyOCR

                        ocr_results = ocr_reader.readtext(cropped_cell_np)
                        if ocr_results:
                            text = " ".join([res_text for _, res_text, _ in ocr_results])
                            current_row_texts.append(text)
                        else:
                            current_row_texts.append("")
                    
                    if current_row_texts: # Chỉ thêm nếu hàng có text (hoặc ô trống)
                        ocr_data[r_idx] = current_row_texts
                        if len(current_row_texts) > max_cols:
                            max_cols = len(current_row_texts)
                
                # Pad các hàng thiếu cột
                for r_key in ocr_data:
                    while len(ocr_data[r_key]) < max_cols:
                        ocr_data[r_key].append("")
                return ocr_data, max_cols

            # Truyền EASYOCR_READER và pil_crop_image_for_structure (ảnh PIL gốc của crop)
            extracted_data_dict, num_cols = apply_ocr_to_cells(cell_coords_for_ocr, EASYOCR_READER, pil_crop_image_for_structure)

            if extracted_data_dict:
                print(f"\n--- Dữ liệu bảng {processed_table_count} trích xuất (số cột: {num_cols}) ---")
                df = pd.DataFrame.from_dict(extracted_data_dict, orient='index')
                # Đặt tên cột nếu muốn: df.columns = [f"Cột {j+1}" for j in range(num_cols)]
                print(df.to_string()) # In ra console

                #---- Lưu CSV ---
                try:
                    root_tk = Tk()
                    root_tk.withdraw()
                    default_csv_name = f"{Path(output_dir_prefix).name}_table_{processed_table_count:02d}_iou{iou_score_val:.2f}.csv"
                    csv_file_path = filedialog.asksaveasfilename(
                        initialfile=default_csv_name,
                        defaultextension=".csv",
                        filetypes=[("CSV files", "*.csv"), ("All files", "*.*")],
                        title="Chọn nơi lưu file CSV cho bảng"
                    )
                    if csv_file_path:
                        df.to_csv(csv_file_path, index=False, encoding='utf-8-sig') # encoding tốt cho tiếng Việt
                        print(f"    - Đã lưu dữ liệu bảng vào: {csv_file_path}")
                    else:
                        print("    - Lưu CSV bị hủy.")
                    root_tk.destroy() # Quan trọng: đóng cửa sổ Tkinter ẩn
                except Exception as e_csv:
                    print(f"    - LỖI khi lưu CSV: {e_csv}")
            else:
                print(f"    - Không có dữ liệu nào được trích xuất từ bảng {processed_table_count}.")


            # Lưu ảnh crop đã xử lý (ảnh bảng trực quan)
            safe_prefix_filename = "".join(c if c.isalnum() or c in ('_', '-') else '_' for c in Path(output_dir_prefix).name)
            output_table_filename = f"{safe_prefix_filename}_table_CVmatchTT_{processed_table_count:02d}_iou{iou_score_val:.2f}.png"
            output_table_path = Path(Path(output_dir_prefix).parent) / output_table_filename

            try:
                if cv2.imwrite(str(output_table_path), final_processed_crop_for_saving_img):
                    saved_crop_paths_list.append(str(output_table_path))
                    print(f"    - Đã lưu ảnh bảng đã xử lý vào: {output_table_path}")
                else:
                    print(f"    - LỖI: Không thể lưu ảnh bảng (cv2.imwrite trả về False): {output_table_path}")
            except Exception as e_save_table:
                print(f"    - LỖI: Ngoại lệ khi lưu ảnh bảng {output_table_path}: {e_save_table}")
        else:
            print(f"    - Xử lý: Crop cuối cùng cho cặp {pair_index+1} không hợp lệ hoặc rỗng. Không được lưu.")
    if SHOW_PROCESSING_IMAGES:
        cv2.destroyAllWindows() # Đóng tất cả cửa sổ OpenCV khi xong một trang
    return saved_crop_paths_list


In [ ]:
def process_pdf_for_tables_pipeline(
    pdf_file_path_main:str|Path, base_output_dir_prefix:str, pdf_processing_dpi:int, enable_pdf_page_deskew:bool,
    pdf_page_deskew_angle_thresh:float, cv_detection_config_main:dict, table_deskew_config_main:dict,
    iou_matching_thresh_config:float, tt_page_confidence_thresh_config_val:float, temp_pdf_pages_dir_path:str|Path,
    # Truyền các model đã load
    tt_detection_model_loaded, tt_detection_processor_loaded,
    tt_structure_model_loaded, easyocr_reader_loaded

) -> tuple[dict[int,list[str]],int]:

    all_page_processing_results:dict[int,list[str]]={}; total_tables_extracted_count=0
    pdf_path_obj=Path(pdf_file_path_main)
    temp_pages_dir_obj=Path(temp_pdf_pages_dir_path)

    shutil.rmtree(temp_pages_dir_obj, ignore_errors=True)
    os.makedirs(temp_pages_dir_obj, exist_ok=True)
    output_main_dir=Path(base_output_dir_prefix).parent
    if not output_main_dir.exists(): os.makedirs(output_main_dir, exist_ok=True)

    pdf_page_pil_images_list:list[Image.Image]=[]
    try:
        # print(f"  Đang chuyển đổi PDF '{pdf_path_obj.name}' (DPI:{pdf_processing_dpi})...")
        start_conversion_time=time.time(); cpu_cores_count=os.cpu_count()or 4; thread_count_val=max(1,cpu_cores_count//2)
        # Kiểm tra poppler_path cho Windows nếu cần
        poppler_path_env = os.getenv("POPPLER_PATH") # Ví dụ: C:/path/to/poppler-xxx/bin
        
        pdf_page_pil_images_list = convert_from_path(
            pdf_path_obj, dpi=pdf_processing_dpi, fmt='png',
            thread_count=thread_count_val, use_pdftocairo=True,
            poppler_path=poppler_path_env # Sẽ là None nếu không đặt, pdf2image tự tìm
        )
        conversion_duration_sec=time.time()-start_conversion_time
        # print(f"  Chuyển đổi PDF hoàn tất: {len(pdf_page_pil_images_list)} trang trong {conversion_duration_sec:.2f} giây.")
        if not pdf_page_pil_images_list: raise ValueError("Chuyển đổi PDF cho ra 0 hình ảnh.")
    except Exception as e_pdf_convert:
        print(f"LỖI NGHIÊM TRỌNG: Chuyển đổi PDF sang ảnh thất bại: {e_pdf_convert}")
        if "poppler" in str(e_pdf_convert).lower() and poppler_path_env is None:
             print(" gợi ý: Nếu trên Windows, hãy thử đặt biến môi trường POPPLER_PATH trỏ đến thư mục bin của Poppler.")
        shutil.rmtree(temp_pages_dir_obj, ignore_errors=True); return {}, 0

    for current_page_idx, current_pil_image in enumerate(pdf_page_pil_images_list):
        current_page_num = current_page_idx + 1
        temp_page_image_file_path:Path|None=None
        # print(f"\nĐang xử lý Trang {current_page_num}/{len(pdf_page_pil_images_list)}...")
        try:
            cv_bgr_page_image = cv2.cvtColor(np.array(current_pil_image), cv2.COLOR_RGB2BGR)
            if cv_bgr_page_image is None or cv_bgr_page_image.size == 0:
                # print(f"  Cảnh báo: Trang {current_page_num} rỗng. Bỏ qua."); 
                all_page_processing_results[current_page_num] = []; continue
            image_to_process_for_saving = cv_bgr_page_image.copy()
            if enable_pdf_page_deskew:
                deskewed_page_version = deskew_page_basic(cv_bgr_page_image, pdf_page_deskew_angle_thresh)
                image_to_process_for_saving = deskewed_page_version
            temp_page_image_filename = f"page_{current_page_num:03d}.png"
            temp_page_image_file_path = temp_pages_dir_obj / temp_page_image_filename
            if not cv2.imwrite(str(temp_page_image_file_path), image_to_process_for_saving):
                # print(f"  Cảnh báo: Không thể lưu ảnh tạm cho trang {current_page_num}. Bỏ qua."); 
                all_page_processing_results[current_page_num] = []; continue
            current_page_output_prefix = f"{base_output_dir_prefix}_pg{current_page_num:03d}"
            saved_table_paths_for_this_page = detect_tables_cv_tt_combined(
                page_image_path=temp_page_image_file_path, output_dir_prefix=current_page_output_prefix,
                cv_detect_params=cv_detection_config_main, table_deskew_params=table_deskew_config_main,
                iou_match_threshold=iou_matching_thresh_config, tt_page_confidence_threshold=tt_page_confidence_thresh_config_val,
                tt_detection_model=tt_detection_model_loaded, tt_detection_processor=tt_detection_processor_loaded,
                tt_structure_model=tt_structure_model_loaded, easyocr_reader=easyocr_reader_loaded
            )
            all_page_processing_results[current_page_num] = saved_table_paths_for_this_page
            num_tables_on_this_page = len(saved_table_paths_for_this_page)
            total_tables_extracted_count += num_tables_on_this_page
            # print(f"  Trang {current_page_num}: Tìm thấy và đã lưu {num_tables_on_this_page} bảng khớp tiêu chí.")
        except KeyboardInterrupt:
            print(f"\n!!! Người dùng đã ngắt xử lý trên trang {current_page_num}. Đang dừng.")
            shutil.rmtree(temp_pages_dir_obj, ignore_errors=True); raise
        except Exception as e_process_page:
            print(f"  LỖI NGHIÊM TRỌNG khi xử lý trang {current_page_num}: {e_process_page}")
            import traceback; traceback.print_exc()
            all_page_processing_results[current_page_num] = []
        finally:
            if temp_page_image_file_path and temp_page_image_file_path.exists():
                try: os.remove(temp_page_image_file_path)
                except OSError as e_remove_temp: print(f"  Cảnh báo: Không thể xóa tệp trang tạm {temp_page_image_file_path}: {e_remove_temp}")
    shutil.rmtree(temp_pages_dir_obj, ignore_errors=True)
    return all_page_processing_results, total_tables_extracted_count


In [ ]:
def main():
    global yolo_model_global, TT_DETECTION_MODEL, TT_DETECTION_PROCESSOR, TT_STRUCTURE_MODEL, EASYOCR_READER # Để truy cập các model đã load

    # Tham số cấu hình
    CONFIG = {
        "PDF_DPI":200,
        "ENABLE_PAGE_DESKEW":True,
        "PAGE_DESKEW_ANGLE_THRESHOLD":0.5,
        "CV_DETECT_PARAMS":{
            'min_line_ratio':0.015, 'min_area_ratio':0.005, 'min_dim_px':20,
            'dilate_iter':2, 'padding':10, 'overlap_threshold_ratio':0.65
        },
        "TABLE_DESKEW_PARAMS":{
            'morph_kernel_width_factor':0.12, 'morph_iterations':1, 'hough_threshold':20,
            'hough_min_line_length_factor':0.15, 'hough_max_line_gap_factor':0.04,
            'canny_low_thresh':40, 'canny_high_thresh':120, 'angle_filter_degrees':20.0,
            'rotation_threshold_degrees':0.20
        },
        "IOU_MATCH_THRESHOLD":0.35,
        "TT_PAGE_CONFIDENCE_THRESHOLD":0.9,
        "OUTPUT_BASE_DIR":Path("./table_extraction_output_local"), # Thay đổi đường dẫn output
        "TEMP_PDF_PAGES_DIR":Path("./temp_pdf_pages_local"),    # Thay đổi đường dẫn temp
        "YOLO_ENABLE_VALIDATION":False,
        "YOLO_MODEL_PATH":"keremberke/yolov8n-table-detection",
    }

    if CONFIG["YOLO_ENABLE_VALIDATION"]:
        try:
            from ultralytics import YOLO # Đảm bảo import
            yolo_model_global, _ = load_yolo_model_safe(CONFIG["YOLO_MODEL_PATH"], yolo_device_global)
            # if yolo_model_global: print("Mô hình YOLO đã tải (cho validation tùy chọn).")
        except ImportError:
             print("Ultralytics chưa được cài. Không thể tải YOLO.")
        except Exception as e:
            print(f"Lỗi tải YOLO: {e}")


    shutil.rmtree(CONFIG["OUTPUT_BASE_DIR"],ignore_errors=True)
    shutil.rmtree(CONFIG["TEMP_PDF_PAGES_DIR"],ignore_errors=True)
    # os.makedirs(CONFIG["OUTPUT_BASE_DIR"],exist_ok=True)

    root = Tk()
    root.withdraw() # Ẩn cửa sổ Tkinter chính
    uploaded_pdf_file_path_main = filedialog.askopenfilename(
        title="Chọn một file PDF để xử lý",
        filetypes=[("PDF files", "*.pdf"), ("All files", "*.*")]
    )
    root.destroy() 

    if not uploaded_pdf_file_path_main:
        sys.exit("Thực thi bị dừng: Không có PDF được cung cấp.")
    else:
        uploaded_pdf_file_path_main = Path(uploaded_pdf_file_path_main)

    if uploaded_pdf_file_path_main and uploaded_pdf_file_path_main.exists():
        # Kiểm tra xem các model cần thiết đã được tải chưa
        if not TT_DETECTION_MODEL or not TT_DETECTION_PROCESSOR or \
           not TT_STRUCTURE_MODEL or not EASYOCR_READER:
            # print("Lỗi nghiêm trọng: Một hoặc nhiều model chính (Table Transformer Detection/Structure, EasyOCR) chưa được tải thành công.")
            # print("Vui lòng kiểm tra thông báo lỗi ở trên và đảm bảo các model có thể được tải về.")
            sys.exit("Thoát do thiếu model.")

        pdf_basename_val=uploaded_pdf_file_path_main.name
        pdf_name_no_ext_val=uploaded_pdf_file_path_main.stem
        current_pdf_output_base_prefix = str(CONFIG["OUTPUT_BASE_DIR"] / pdf_name_no_ext_val)

        processing_start_main_time = time.time()
        try:
            page_results_dict, total_tables_extracted_final_count = process_pdf_for_tables_pipeline(
                pdf_file_path_main=uploaded_pdf_file_path_main,
                base_output_dir_prefix=current_pdf_output_base_prefix,
                pdf_processing_dpi=CONFIG["PDF_DPI"],
                enable_pdf_page_deskew=CONFIG["ENABLE_PAGE_DESKEW"],
                pdf_page_deskew_angle_thresh=CONFIG["PAGE_DESKEW_ANGLE_THRESHOLD"],
                cv_detection_config_main=CONFIG["CV_DETECT_PARAMS"],
                table_deskew_config_main=CONFIG["TABLE_DESKEW_PARAMS"],
                iou_matching_thresh_config=CONFIG["IOU_MATCH_THRESHOLD"],
                tt_page_confidence_thresh_config_val=CONFIG["TT_PAGE_CONFIDENCE_THRESHOLD"],
                temp_pdf_pages_dir_path=CONFIG["TEMP_PDF_PAGES_DIR"],
                # Truyền các model đã load
                tt_detection_model_loaded=TT_DETECTION_MODEL,
                tt_detection_processor_loaded=TT_DETECTION_PROCESSOR,
                tt_structure_model_loaded=TT_STRUCTURE_MODEL,
                easyocr_reader_loaded=EASYOCR_READER
            )
            processing_duration_main_sec = time.time() - processing_start_main_time
        except KeyboardInterrupt: print("\n!!! XỬ LÝ BỊ NGƯỜI DÙNG NGẮT !!!")
        except Exception as e_main_process:
            print(f"\n!!! ĐÃ XẢY RA LỖI NGHIÊM TRỌNG: {e_main_process} !!!")
            import traceback; traceback.print_exc()
        finally:
            if SHOW_PROCESSING_IMAGES:
                cv2.destroyAllWindows()
    else:
        print("\nLỗi: Đường dẫn tệp PDF không hợp lệ hoặc tệp không tồn tại.")

    overall_script_duration_sec = time.time() - overall_start_time
    # print(f"\nTổng thời gian thực thi script: {overall_script_duration_sec:.2f} giây. --- Kết thúc Script ---")

if __name__ == "__main__":
    main()